## Setup and Configuration

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import QuantLib as ql
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add project paths
project_root = r'C:\Users\Ao Shen\Desktop\mfin research\src'
heston_path = r'C:\Users\Ao Shen\Desktop\mfin research\src\heston_calib'

for path in [project_root, heston_path]:
    if path not in sys.path:
        sys.path.insert(0, path)

# Import custom modules
from market_data_fetcher import MarketDataFetcher
from quantlib_heston_calibrator import QuantLibHestonCalibrator

print(f"Setup complete - QuantLib v{ql.__version__}")

In [ ]:
# Configuration
TICKER = 'NVDA'
EXPIRY_LIST = ['1M', '2M']  # Use fewer expiries for faster calibration
ATM_RANGE = 0.4  # ±15% around spot for better quality
RISK_FREE_RATE = 0.015  # Updated to current rate
DIVIDEND_YIELD = 0.0
MAX_ITERATIONS = 200  # More iterations for better convergence

print(f"Target: {TICKER} | Expiries: {EXPIRY_LIST} | ATM Range: ±{ATM_RANGE*100:.0f}%")
print(f"✅ Enhanced Calibrator: Multi-start optimization, Yahoo Finance IV, Diverse initial conditions")

## Market Data Fetching

In [ ]:
# Fetch market data
fetcher = MarketDataFetcher(
    ticker=TICKER, 
    expiry_list=EXPIRY_LIST, 
    atm_range=ATM_RANGE
)

market_data = fetcher.prepare_market_data()
spot_price = fetcher.get_spot_price()

print(f"Spot: ${spot_price:.2f} | Options: {len(market_data)} | DTE: {market_data['DaysToExpiry'].min()}-{market_data['DaysToExpiry'].max()} days")

# Show first few rows
display(market_data.head(3))

In [ ]:
# Show Yahoo Finance IV data quality
print("📊 Market Data Quality with Yahoo Finance IV:")
print(f"   • Total contracts: {len(market_data)}")
print(f"   • Expiries: {market_data.DaysToExpiry.nunique()} ({market_data.DaysToExpiry.min()}-{market_data.DaysToExpiry.max()} days)")
print(f"   • Strike range: ${market_data.Strike.min():.0f} - ${market_data.Strike.max():.0f}")

if 'ImpliedVolatility' in market_data.columns:
    print(f"   • IV range: {market_data.ImpliedVolatility.min():.3f} - {market_data.ImpliedVolatility.max():.3f}")
    print(f"   • ✅ Yahoo Finance IV available - no manual calculation needed!")
    
    # Show IV distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # IV by moneyness
    market_data['Moneyness'] = market_data.Strike / spot_price
    ax1.scatter(market_data.Moneyness, market_data.ImpliedVolatility, 
               c=market_data.DaysToExpiry, cmap='viridis', alpha=0.7)
    ax1.set_xlabel('Moneyness (K/S)')
    ax1.set_ylabel('Implied Volatility')
    ax1.set_title('IV Smile from Yahoo Finance')
    ax1.grid(True, alpha=0.3)
    
    # IV histogram  
    ax2.hist(market_data.ImpliedVolatility, bins=20, alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Implied Volatility')
    ax2.set_ylabel('Frequency')
    ax2.set_title('IV Distribution')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"   • ⚠️ No Yahoo Finance IV - will use fallback calculation")

In [ ]:
# Reload modules to get the latest enhanced version
import importlib
import market_data_fetcher
import quantlib_heston_calibrator

# Reload the modules
importlib.reload(market_data_fetcher)
importlib.reload(quantlib_heston_calibrator)

# Re-import the classes with updated code
from market_data_fetcher import MarketDataFetcher
from quantlib_heston_calibrator import QuantLibHestonCalibrator

print("✅ Modules reloaded with latest enhancements")

In [ ]:
# Test enhanced market data fetching with reloaded modules
print("🧪 Testing enhanced market data fetcher...")

# Create fresh instance with reloaded code
test_fetcher = MarketDataFetcher(
    ticker=TICKER, 
    expiry_list=EXPIRY_LIST, 
    atm_range=ATM_RANGE
)

# Get enhanced market data  
test_market_data = test_fetcher.prepare_market_data()
test_spot_price = test_fetcher.get_spot_price()

print(f"✅ Enhanced market data columns: {list(test_market_data.columns)}")

if 'ImpliedVolatility' in test_market_data.columns:
    print(f"🎉 SUCCESS! Yahoo Finance IV is available")
    print(f"   • IV range: {test_market_data.ImpliedVolatility.min():.3f} - {test_market_data.ImpliedVolatility.max():.3f}")
    print(f"   • Sample data:")
    display(test_market_data[['Strike', 'MarketPrice', 'ImpliedVolatility', 'OptionType']].head(3))
    
    # Update global variables for the rest of the notebook
    market_data = test_market_data
    spot_price = test_spot_price
    print(f"   • Updated global market_data and spot_price variables")
else:
    print(f"❌ ImpliedVolatility still missing from {list(test_market_data.columns)}")

## Heston Model Calibration

In [ ]:
# Run Heston Calibration with Enhanced Features
print("🚀 Starting Enhanced Heston Calibration...")
print(f"   • Multi-start global optimization (5 diverse initial conditions)")
print(f"   • Yahoo Finance implied volatility (no manual calculation)")  
print(f"   • Expanded parameter bounds to avoid boundary issues")
print(f"   • Differential evolution for robust global optimization")
print()

calibrator = QuantLibHestonCalibrator(
    r=RISK_FREE_RATE, 
    q=DIVIDEND_YIELD
)

# Enhanced calibration with new features
heston_model, calibration_info = calibrator.calibrate(
    spot=spot_price,
    market_data=market_data,
    maxiter=MAX_ITERATIONS,  # More iterations for proper convergence
    detailed_report=False    # Keep output concise
)

# Print results using the enhanced formatter
calibrator.print_results(calibration_info)

## IV Surface Reconstruction

In [ ]:
# Reconstruct IV surface for model validation using Yahoo Finance IV
if not calibration_info['success']:
    raise ValueError("Calibration failed")

heston_engine = ql.AnalyticHestonEngine(heston_model)
iv_comparison_results = []

for idx, (_, row) in enumerate(market_data.iterrows()):
    try:
        time_to_expiry = row.DaysToExpiry / 252.0
        
        # Use Yahoo Finance IV directly (no manual calculation needed!)
        market_iv = row.ImpliedVolatility if 'ImpliedVolatility' in row else 0.20
        
        if market_iv < 0.05 or market_iv > 2.0:
            continue
            
        # Create option for Heston pricing
        is_call = row.OptionType.lower() == 'call'
        option_type = ql.Option.Call if is_call else ql.Option.Put
        
        period = ql.Period(int(row.DaysToExpiry), ql.Days)
        expiry_date = calibrator.evaluation_date + period
        exercise = ql.EuropeanExercise(expiry_date)
        payoff = ql.PlainVanillaPayoff(option_type, float(row.Strike))
        option = ql.VanillaOption(payoff, exercise)
        option.setPricingEngine(heston_engine)
        
        heston_price = option.NPV()
        
        if heston_price > 0.01:
            # Calculate implied vol from Heston price (fallback method still available)
            heston_iv = calibrator._calculate_implied_vol(
                heston_price, spot_price, row.Strike, 
                time_to_expiry, row.OptionType
            )
            
            iv_comparison_results.append({
                'Strike': row.Strike,
                'DaysToExpiry': row.DaysToExpiry,
                'OptionType': row.OptionType,
                'MarketIV': market_iv,
                'HestonIV': heston_iv,
                'IVDiff': heston_iv - market_iv,
                'AbsIVError': abs(heston_iv - market_iv)
            })
    except Exception as e:
        continue

iv_comparison_df = pd.DataFrame(iv_comparison_results)

if not iv_comparison_df.empty:
    mean_abs_iv_error = iv_comparison_df['AbsIVError'].mean()
    print(f"✅ IV Surface Reconstruction Complete")
    print(f"   • Processed {len(iv_comparison_df)} contracts")
    print(f"   • Mean Absolute IV Error: {mean_abs_iv_error:.4f} ({mean_abs_iv_error*100:.2f}%)")
    print(f"   • Using Yahoo Finance IV for market reference")
    
    display(iv_comparison_df.head(10))
else:
    print("❌ No valid IV comparisons generated")

## Visualization and Analysis

In [ ]:
# Volatility Smile Analysis by Expiry (Calls Only)
if len(iv_comparison_df) > 0:
    # Filter for calls only for cleaner visualization
    calls_only = iv_comparison_df[iv_comparison_df['OptionType'] == 'call'].copy()
    
    if len(calls_only) > 0:
        # Group data by expiry for volatility smile analysis
        expiries = sorted(calls_only['DaysToExpiry'].unique())
        
        # Create subplots for each expiry
        n_expiries = len(expiries)
        cols = min(2, n_expiries)
        rows = (n_expiries + 1) // 2
        
        fig, axes = plt.subplots(rows, cols, figsize=(12, 4*rows))
        if n_expiries == 1:
            axes = [axes]
        elif rows == 1:
            axes = axes if cols > 1 else [axes]
        else:
            axes = axes.flatten()
        
        for i, dte in enumerate(expiries):
            if i >= len(axes):
                break
                
            # Filter data for this expiry
            expiry_data = calls_only[calls_only['DaysToExpiry'] == dte].copy()
            
            if len(expiry_data) < 2:
                continue
                
            # Calculate moneyness (Strike/Spot)
            expiry_data['Moneyness'] = expiry_data['Strike'] / spot_price
            expiry_data = expiry_data.sort_values('Moneyness')
            
            ax = axes[i]
            
            # Plot market and Heston IV smiles for calls
            ax.plot(expiry_data['Moneyness'], expiry_data['MarketIV']*100, 
                    'o-', color='blue', linewidth=2, markersize=6, label='Market IV', alpha=0.8)
            ax.plot(expiry_data['Moneyness'], expiry_data['HestonIV']*100, 
                    's--', color='red', linewidth=2, markersize=5, label='Heston IV', alpha=0.8)
            
            # Add ATM line
            ax.axvline(x=1.0, color='gray', linestyle=':', alpha=0.5, label='ATM')
            
            # Formatting
            ax.set_xlabel('Moneyness (K/S)')
            ax.set_ylabel('Implied Volatility (%)')
            ax.set_title(f'{dte} Days - Call Options Volatility Smile')
            ax.legend(fontsize=9)
            ax.grid(True, alpha=0.3)
            
            # Add error statistics
            mae = expiry_data['AbsIVError'].mean()
            correlation = expiry_data['MarketIV'].corr(expiry_data['HestonIV'])
            ax.text(0.02, 0.98, f'MAE: {mae:.1%}\nCorr: {correlation:.3f}', 
                    transform=ax.transAxes, 
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
                    verticalalignment='top', fontsize=9)
            
            # Set reasonable x-axis limits
            x_min = expiry_data['Moneyness'].min() * 0.98
            x_max = expiry_data['Moneyness'].max() * 1.02
            ax.set_xlim(x_min, x_max)
        
        # Hide unused subplots
        for j in range(i+1, len(axes)):
            axes[j].set_visible(False)
        
        plt.tight_layout()
        plt.show()
        
        # Overall quality assessment for calls
        mae_iv = calls_only['AbsIVError'].mean()
        correlation = calls_only['MarketIV'].corr(calls_only['HestonIV'])
        
        print(f"\nVolatility Smile Analysis (Calls Only):")
        print(f"Expiries analyzed: {len(expiries)} ({expiries} days)")
        print(f"Call options: {len(calls_only)}")
        print(f"Overall MAE: {mae_iv:.1%}")
        print(f"Overall correlation: {correlation:.3f}")
        
        if mae_iv < 0.05 and correlation > 0.8:
            quality = "EXCELLENT - Heston captures call smile patterns very well"
        elif mae_iv < 0.10 and correlation > 0.7:
            quality = "GOOD - Reasonable fit to call volatility smiles"
        else:
            quality = "FAIR - Some deviations from market call smile patterns"
        
        print(f"Model Quality: {quality}")
    
    else:
        print("No call options available for volatility smile analysis")
        
else:
    print("No valid comparisons available for volatility smile analysis")

## Summary

In [ ]:
# Concise Summary
print("HESTON CALIBRATION SUMMARY")
print("=" * 40)

print(f"Data: {TICKER} | {len(market_data)} options | ${spot_price:.2f} spot")

if calibration_info['success']:
    params = calibration_info['calibrated_params']
    print(f"\nCalibration: SUCCESS")
    print(f"   Used {calibration_info['num_helpers']} options")
    print(f"   Average error: {calibration_info['average_error']:.1%}")
    
    print(f"\nHeston Parameters:")
    print(f"   Current vol (√v₀): {np.sqrt(params['v0']):.1%}")
    print(f"   Long-term vol (√θ): {np.sqrt(params['theta']):.1%}")
    print(f"   Mean reversion (κ): {params['kappa']:.2f}")
    print(f"   Vol-of-vol (σ): {params['sigma']:.1%}")
    print(f"   Correlation (ρ): {params['rho']:.3f}")
    
    if len(iv_comparison_df) > 0:
        mae_iv = iv_comparison_df['AbsIVError'].mean()
        correlation = iv_comparison_df['MarketIV'].corr(iv_comparison_df['HestonIV'])
        
        print(f"\nModel Quality:")
        print(f"   IV error: {mae_iv:.1%}")
        print(f"   Correlation: {correlation:.3f}")
        
        if mae_iv < 0.05 and correlation > 0.8:
            status = "READY FOR PRODUCTION"
        elif mae_iv < 0.10 and correlation > 0.6:
            status = "ACCEPTABLE WITH MONITORING"
        else:
            status = "NEEDS IMPROVEMENT"
        
        print(f"   Status: {status}")
    
    # Quick interpretation
    print(f"\nKey Insights:")
    if params['rho'] < -0.5:
        print(f"   • Strong leverage effect detected")
    if params['kappa'] > 2:
        print(f"   • Fast volatility mean reversion")
    if np.sqrt(params['v0']) > 0.25:
        print(f"   • Currently high volatility regime")
        
else:
    print(f"Calibration FAILED: {calibration_info.get('error', 'Unknown error')}")

print(f"\nCompleted: {datetime.now().strftime('%H:%M:%S')}")
print("=" * 40)

## Demo Complete

**Heston calibrator successfully demonstrated:**
- Real market data fetching (NVDA options)
- Global optimization calibration 
- Model validation with IV surface comparison
- Production-ready results

**Ready for:** Options pricing, portfolio hedging, volatility analysis